In [1]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import pandas as pd
import numpy as np
import itertools
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.pylab as pit
from matplotlib.pylab import rcParams
%matplotlib inline
from matplotlib import style
style.use('ggplot')
rcParams['figure.figsize'] = 20,10
import re

C:\Users\archana.parihar\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:

dfPairs = pd.DataFrame()
dfPairs = pd.read_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_2_Medoid_Cluster_2.csv')
ListOfPairsToCheck = dfPairs['Pairs'].tolist()

In [3]:
df = pd.DataFrame()
df = pd.read_csv('C:/Users/archana.parihar/Downloads/allStocks16-17-18.csv')
ListOfSnPCompanies = df['Name'].tolist()
ListOfSnPCompanies = list(set(ListOfSnPCompanies))
print(ListOfSnPCompanies)

['COG', 'NFLX', 'DXC', 'SYK', 'BA', 'LEG', 'TEL', 'MCHP', 'ABBV', 'PPL', 'APA', 'SPGI', 'MCO', 'FAST', 'XLNX', 'ATVI', 'HCP', 'FOX', 'CF', 'JCI', 'JEC', 'BWA', 'CERN', 'STZ', 'JBHT', 'GM', 'HRL', 'EQT', 'GPN', 'GIS', 'CHRW', 'WU', 'FISV', 'CRM', 'PXD', 'SCG', 'MMM', 'PEP', 'TRV', 'HAS', 'DWDP', 'FIS', 'SRE', 'NVDA', 'BMY', 'IQV', 'MMC', 'UAA', 'VTR', 'ADP', 'DHI', 'LNT', 'GOOGL', 'CAG', 'MOS', 'ETR', 'AFL', 'DG', 'GD', 'T', 'LUV', 'KO', 'ALK', 'DISCA', 'AAP', 'AWK', 'REGN', 'GE', 'VRSN', 'NTRS', 'ULTA', 'ES', 'KHC', 'CTL', 'LEN', 'FOXA', 'WMT', 'OXY', 'GOOG', 'TSS', 'HAL', 'HOG', 'SJM', 'UHS', 'GPS', 'PBCT', 'MPC', 'NWSA', 'ORCL', 'RMD', 'QRVO', 'CXO', 'CVX', 'CTSH', 'CB', 'EMN', 'AYI', 'MAS', 'PNW', 'HON', 'NUE', 'CSCO', 'XOM', 'AMG', 'UA', 'HD', 'MDLZ', 'TJX', 'ANTM', 'COL', 'MAA', 'UNH', 'WYN', 'EQR', 'IBM', 'MTB', 'DVN', 'LLL', 'FTI', 'NEE', 'RF', 'MSI', 'GILD', 'HSIC', 'APC', 'NOC', 'FRT', 'ICE', 'HPE', 'HES', 'AVY', 'CELG', 'DVA', 'GWW', 'BSX', 'DTE', 'DFS', 'VIAB', 'MRO', 'SRCL'

In [4]:
for i in range(0,len(ListOfSnPCompanies)):
    if ListOfSnPCompanies[i]=='BRK.B':
        ListOfSnPCompanies[i] = 'BRKB'
    if ListOfSnPCompanies[i]=='BF.B':
        ListOfSnPCompanies[i] = 'BFB'
print(ListOfSnPCompanies)

['COG', 'NFLX', 'DXC', 'SYK', 'BA', 'LEG', 'TEL', 'MCHP', 'ABBV', 'PPL', 'APA', 'SPGI', 'MCO', 'FAST', 'XLNX', 'ATVI', 'HCP', 'FOX', 'CF', 'JCI', 'JEC', 'BWA', 'CERN', 'STZ', 'JBHT', 'GM', 'HRL', 'EQT', 'GPN', 'GIS', 'CHRW', 'WU', 'FISV', 'CRM', 'PXD', 'SCG', 'MMM', 'PEP', 'TRV', 'HAS', 'DWDP', 'FIS', 'SRE', 'NVDA', 'BMY', 'IQV', 'MMC', 'UAA', 'VTR', 'ADP', 'DHI', 'LNT', 'GOOGL', 'CAG', 'MOS', 'ETR', 'AFL', 'DG', 'GD', 'T', 'LUV', 'KO', 'ALK', 'DISCA', 'AAP', 'AWK', 'REGN', 'GE', 'VRSN', 'NTRS', 'ULTA', 'ES', 'KHC', 'CTL', 'LEN', 'FOXA', 'WMT', 'OXY', 'GOOG', 'TSS', 'HAL', 'HOG', 'SJM', 'UHS', 'GPS', 'PBCT', 'MPC', 'NWSA', 'ORCL', 'RMD', 'QRVO', 'CXO', 'CVX', 'CTSH', 'CB', 'EMN', 'AYI', 'MAS', 'PNW', 'HON', 'NUE', 'CSCO', 'XOM', 'AMG', 'UA', 'HD', 'MDLZ', 'TJX', 'ANTM', 'COL', 'MAA', 'UNH', 'WYN', 'EQR', 'IBM', 'MTB', 'DVN', 'LLL', 'FTI', 'NEE', 'RF', 'MSI', 'GILD', 'HSIC', 'APC', 'NOC', 'FRT', 'ICE', 'HPE', 'HES', 'AVY', 'CELG', 'DVA', 'GWW', 'BSX', 'DTE', 'DFS', 'VIAB', 'MRO', 'SRCL'

In [5]:
dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""
count = 0

#Making the dat dictionary 
for c in ListOfSnPCompanies:
    dfcomp = df.loc[df['Name'] == c]
    listComp = dfcomp['close'].values
    dat[c] = listComp

print(dat['BRKB'])
print(dat['BFB'])

[130.75 131.25 131.33 129.48 128.33 128.36 128.8  128.66 125.7  126.24
 127.44 126.56 126.78 127.72 126.58 125.09 128.07 136.94 136.73 136.84
 137.97 138.72 137.8  139.13 138.62 140.65 143.79 143.22 141.2  141.7
 140.81 141.06 140.99 142.4  146.89 144.83 144.24 143.88 144.62 142.91
 144.78 142.87 143.31 141.99 142.11 141.14 141.82 141.41 142.53 142.23
 141.76 143.96 142.23 142.51 141.91 143.65 143.93 144.67 143.93 143.31
 143.63 143.12 145.65 145.56 147.22 146.69 147.74 147.72 150.35 150.72
 150.43 149.58 149.78 146.62 148.97 143.76 143.17 144.15 143.87 144.52
 145.2  143.95 144.91 143.63 143.19 143.64 142.95 146.91 147.79 151.28
 156.06 156.92 160.12 159.39 160.21 161.34 164.69 164.11 164.86 166.12
 163.83 164.08 163.3  163.41 162.02 161.47 162.23 161.41 163.61 163.
 163.94 163.42 162.93 162.87 164.04 164.01 177.28 175.89 175.68 175.4
 175.3  174.97 175.37 174.98 166.87 166.86 165.83 166.47 166.17 166.35
 165.7  165.11 165.44 166.05 166.73 166.34 166.55 165.02 163.46 163.72
 163.27 16

In [6]:
listOfPairsFinal = []
for pair in ListOfPairsToCheck:
    pair = pair.replace('(',"")
    pair = pair.replace(')',"")
    pair = pair.replace(" ", "")
    pair = pair.split(',')
    pair[0] = re.sub('[^A-Za-z0-9]+', '', pair[0])    
    pair[1] = re.sub('[^A-Za-z0-9]+', '', pair[1])
    listOfPairsFinal.append(pair)

print(listOfPairsFinal[20])
print(listOfPairsFinal[20][1])
print(dat[listOfPairsFinal[20][1]][5])
#print(dat['RMD'])
print(listOfPairsFinal)
print(len(listOfPairsFinal))

['ABC', 'EBAY']
EBAY
26.1
[['AAL', 'HII'], ['AAL', 'KORS'], ['AAP', 'ALK'], ['AAP', 'AMD'], ['AAP', 'APA'], ['AAP', 'CMG'], ['AAP', 'EXPE'], ['AAP', 'ILMN'], ['AAP', 'M'], ['AAP', 'NEM'], ['AAP', 'RCL'], ['AAP', 'STX'], ['AAP', 'UAA'], ['AAP', 'UAL'], ['AAP', 'VFC'], ['AAP', 'WDC'], ['ABC', 'AOS'], ['ABC', 'CF'], ['ABC', 'CNC'], ['ABC', 'DLTR'], ['ABC', 'EBAY'], ['ABC', 'HBI'], ['ABC', 'KMX'], ['ABC', 'KSS'], ['ABC', 'PRGO'], ['ABC', 'SEE'], ['ABC', 'VRTX'], ['ADI', 'AES'], ['ADI', 'AFL'], ['ADI', 'AGN'], ['ADI', 'AJG'], ['ADI', 'ALGN'], ['ADI', 'ALK'], ['ADI', 'ALL'], ['ADI', 'AMG'], ['ADI', 'AON'], ['ADI', 'AOS'], ['ADI', 'APD'], ['ADI', 'APH'], ['ADI', 'ARE'], ['ADI', 'AVY'], ['ADI', 'AZO'], ['ADI', 'BBT'], ['ADI', 'BXP'], ['ADI', 'CHRW'], ['ADI', 'CI'], ['ADI', 'CMA'], ['ADI', 'CMG'], ['ADI', 'CNC'], ['ADI', 'CNP'], ['ADI', 'COF'], ['ADI', 'COG'], ['ADI', 'COL'], ['ADI', 'COO'], ['ADI', 'CTL'], ['ADI', 'CTSH'], ['ADI', 'CTXS'], ['ADI', 'CXO'], ['ADI', 'D'], ['ADI', 'DE'], ['ADI', '

In [7]:
import csv

count = 0
#s_and_p = 
#dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""

for pair in listOfPairsFinal:
    print(pair)
    in_market = 0
    short_pos = 0
    equity_curve = float(100)
    ret = []
    f_ret = []
    a_ret = []
    b_ret = []
    zdiff = []
    signal = [] #shod i long or short
    lCap5 = []
    sCap5 =[]
    age = [0]   #should be -1 indexed
    aged = []
    pos = [0]   #should be -1 indexed .. signal was +1 but long cap has been reached then pos = 0 .. im not in market
    for i in range (5,212):
        a_ret.append((dat[pair[0]][i]-dat[pair[0]][i-5])/dat[pair[0]][i-5])
        b_ret.append((dat[pair[1]][i]-dat[pair[1]][i-5])/dat[pair[1]][i-5])
    for j in range (59,206):
        zdiff.append(a_ret[j+1]/np.std(a_ret[0:j+1])-b_ret[j+1]/np.std(b_ret[0:j+1]))
    for k in range (65,211):
        f_ret.append((dat[pair[0]][k+1]-dat[pair[0]][k])/dat[pair[0]][k]-(dat[pair[1]][k+1]-dat[pair[1]][k])/dat[pair[1]][k])
    for l in range (0,len(zdiff)):
        if zdiff[l] < longEntryThr:
            signal.append(1)
        elif zdiff[l] > shortEntryThr:
            signal.append(-1)
        else:
            signal.append(0)

        if pos[l] == 1 and zdiff[l] > longCap:
            lCap5.append(1)
        else:
            lCap5.append(0)
        if pos[l] == -1 and zdiff[l] < shortCap:
            sCap5.append(1)
        else:
            sCap5.append(0)

        if signal[l]!=0:
            if pos[l] == 0:
                age.append(1)
            else:
                if pos[l]+signal[l] == 0:
                    age.append(1)
                else:
                    #check here if age should be age + 1
                    age.append(1)
        else:
            if age[l] == 5:
                age.append(0)
            else:
                if pos[l] == 1 and lCap5[l] != 0 or pos[l] == -1 and sCap5[l] != 0:
                    age.append(0)
                else:
                    if signal[l] + pos[l] != 0:
                        age.append(age[l] + 1)
                    else:
                        age.append(0)

        if age[l] == 5:
            if signal[l] == 0:
                aged.append(1)
            else:
                aged.append(0)
        else:
            aged.append(0)

        if signal[l] == 0:
            if aged[l] + sCap5[l] + lCap5[l] == 0:
                pos.append(pos[l])
            else:
                if aged[l] + sCap5[l] > 0 and pos[l] == -1:
                    pos.append(0)
                elif aged[l] + lCap5[l] > 0 and pos[l] == 1:
                    pos.append(0)
                else:
                    pos.append(pos[l])
        else:
            if signal[l]!=pos[l]:
                pos.append(signal[l])
            else:
                if aged[l] + sCap5[l] + lCap5[l] == 0:
                    pos.append(signal[l])
                else:
                    if aged[l] + sCap5[l] > 0 and pos[l] == -1  and signal[l] == -1:
                        pos.append(0)
                    elif aged[l] + lCap5[l] > 0 and pos[l] == 1 and signal[l] == 1:
                        pos.append(0)
                    else:
                        pos.append(signal[l])
    count = 0
        
    countOfPosResults = 0
    countOfNegResults = 0
    countOfTotalResults = 0
    
    for m in range(1,len(pos)-1):
        if(pos[m]*f_ret[m-1]>0):
            count = count + 1
        elif (pos[m]*f_ret[m-1]<0):
            count = count -1
        ret.append(pos[m]*f_ret[m-1])
        equity_curve = equity_curve*(1+pos[m]*f_ret[m-1])
        if pos[m]!=0:
            in_market = in_market + 1
        if pos[m]==-1:
            short_pos = short_pos + 1
    details = {}
    details["Sharpe"] = np.sqrt(260)*np.mean(ret)/np.std(ret)
    details["CumRet"] = (equity_curve-100)/100
    details["In_market"] = in_market/440
    details["Short"] = short_pos/440
    done_pair.append((pair[0], pair[1], details))

    if details["Sharpe"]>0 and details["CumRet"]>0:
        pos_result = pos_result  + str(done_pair[-1]) + "\n\n"
    elif details["Sharpe"]<0 and details["CumRet"]<0:
        neg_result = neg_result  + str(done_pair[-1]) + "\n\n"
    else:
        mixed_result = mixed_result  + str(done_pair[-1]) + "\n\n"
    result = result + str(done_pair[-1]) + "\n\n"

file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/analysis_2_Medoids_Cluster_2.txt', 'w')
file.write(result)
file.close()

file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/pos_analysis_2_Medoids_Cluster_2.txt', 'w')
file.write(pos_result)
file.close()

file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/neg_analysis_2_Medoids_Cluster_2.txt', 'w')
file.write(neg_result)
file.close()

file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/mixed_analysis_2_Medoids_Cluster_2.txt', 'w')
file.write(mixed_result)
file.close()



['AAL', 'HII']
['AAL', 'KORS']
['AAP', 'ALK']
['AAP', 'AMD']
['AAP', 'APA']
['AAP', 'CMG']
['AAP', 'EXPE']
['AAP', 'ILMN']
['AAP', 'M']
['AAP', 'NEM']
['AAP', 'RCL']
['AAP', 'STX']
['AAP', 'UAA']
['AAP', 'UAL']
['AAP', 'VFC']
['AAP', 'WDC']
['ABC', 'AOS']
['ABC', 'CF']
['ABC', 'CNC']
['ABC', 'DLTR']
['ABC', 'EBAY']
['ABC', 'HBI']
['ABC', 'KMX']
['ABC', 'KSS']
['ABC', 'PRGO']
['ABC', 'SEE']
['ABC', 'VRTX']
['ADI', 'AES']
['ADI', 'AFL']
['ADI', 'AGN']
['ADI', 'AJG']
['ADI', 'ALGN']
['ADI', 'ALK']
['ADI', 'ALL']
['ADI', 'AMG']
['ADI', 'AON']
['ADI', 'AOS']
['ADI', 'APD']
['ADI', 'APH']
['ADI', 'ARE']
['ADI', 'AVY']
['ADI', 'AZO']
['ADI', 'BBT']
['ADI', 'BXP']
['ADI', 'CHRW']
['ADI', 'CI']
['ADI', 'CMA']
['ADI', 'CMG']
['ADI', 'CNC']
['ADI', 'CNP']
['ADI', 'COF']
['ADI', 'COG']
['ADI', 'COL']
['ADI', 'COO']
['ADI', 'CTL']
['ADI', 'CTSH']
['ADI', 'CTXS']
['ADI', 'CXO']
['ADI', 'D']
['ADI', 'DE']
['ADI', 'DG']
['ADI', 'DGX']
['ADI', 'DISCA']
['ADI', 'DISCK']
['ADI', 'DISH']
['ADI', 'DLR']
['

['AEE', 'DE']
['AEE', 'DG']
['AEE', 'DGX']
['AEE', 'DHI']
['AEE', 'DISH']
['AEE', 'DOV']
['AEE', 'DPS']
['AEE', 'DRE']
['AEE', 'ECL']
['AEE', 'EIX']
['AEE', 'EMN']
['AEE', 'EMR']
['AEE', 'EQR']
['AEE', 'EQT']
['AEE', 'ESRX']
['AEE', 'ESS']
['AEE', 'ETN']
['AEE', 'F']
['AEE', 'FBHS']
['AEE', 'FITB']
['AEE', 'FLR']
['AEE', 'FLS']
['AEE', 'FRT']
['AEE', 'FTI']
['AEE', 'GGP']
['AEE', 'GPC']
['AEE', 'GPN']
['AEE', 'GPS']
['AEE', 'GWW']
['AEE', 'HAS']
['AEE', 'HCA']
['AEE', 'HCP']
['AEE', 'HES']
['AEE', 'HIG']
['AEE', 'HOG']
['AEE', 'HRB']
['AEE', 'HSIC']
['AEE', 'HSY']
['AEE', 'IR']
['AEE', 'IT']
['AEE', 'IVZ']
['AEE', 'JBHT']
['AEE', 'JEC']
['AEE', 'KEY']
['AEE', 'KIM']
['AEE', 'KMB']
['AEE', 'KMX']
['AEE', 'KORS']
['AEE', 'KSS']
['AEE', 'KSU']
['AEE', 'L']
['AEE', 'LB']
['AEE', 'LEN']
['AEE', 'LKQ']
['AEE', 'LLL']
['AEE', 'LYB']
['AEE', 'MAA']
['AEE', 'MAC']
['AEE', 'MAS']
['AEE', 'MAT']
['AEE', 'MET']
['AEE', 'MLM']
['AEE', 'MOS']
['AEE', 'MSI']
['AEE', 'MTB']
['AEE', 'MYL']
['AEE', 'NBL

C:\Users\archana.parihar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:127: RuntimeWarning: invalid value encountered in double_scalars


['AES', 'AVB']
['AES', 'BEN']
['AES', 'CF']
['AES', 'CMA']
['AES', 'CMG']
['AES', 'CMI']
['AES', 'COG']
['AES', 'COO']
['AES', 'CPB']
['AES', 'CXO']
['AES', 'DHI']
['AES', 'DLR']
['AES', 'DPS']
['AES', 'DRE']
['AES', 'EQIX']
['AES', 'EQR']
['AES', 'EQT']
['AES', 'ETN']
['AES', 'EXR']
['AES', 'FBHS']
['AES', 'GPS']
['AES', 'HCA']
['AES', 'HES']
['AES', 'HRL']
['AES', 'ILMN']
['AES', 'KIM']
['AES', 'LNC']
['AES', 'LYB']
['AES', 'MAA']
['AES', 'MCK']
['AES', 'MNST']
['AES', 'NFLX']
['AES', 'NOV']
['AES', 'PRGO']
['AES', 'PSA']
['AES', 'PVH']
['AES', 'PWR']
['AES', 'RCL']
['AES', 'REG']
['AES', 'ROK']
['AES', 'SCG']
['AES', 'STX']
['AES', 'TAP']
['AES', 'TIF']
['AES', 'UHS']
['AES', 'VRSN']
['AES', 'WDC']
['AFL', 'AGN']
['AFL', 'AIV']
['AFL', 'AJG']
['AFL', 'ALB']
['AFL', 'ALGN']
['AFL', 'ALK']
['AFL', 'AME']
['AFL', 'ANSS']
['AFL', 'AON']
['AFL', 'AOS']
['AFL', 'ARE']
['AFL', 'ARNC']
['AFL', 'AVB']
['AFL', 'AWK']
['AFL', 'BLL']
['AFL', 'BXP']
['AFL', 'CA']
['AFL', 'CAG']
['AFL', 'CAH']
['

['AIG', 'DOV']
['AIG', 'DPS']
['AIG', 'DRE']
['AIG', 'DRI']
['AIG', 'DTE']
['AIG', 'DUK']
['AIG', 'DVA']
['AIG', 'EBAY']
['AIG', 'ECL']
['AIG', 'ED']
['AIG', 'EFX']
['AIG', 'EIX']
['AIG', 'EMN']
['AIG', 'EMR']
['AIG', 'EQIX']
['AIG', 'EQR']
['AIG', 'ESRX']
['AIG', 'ESS']
['AIG', 'ETN']
['AIG', 'ETR']
['AIG', 'EXC']
['AIG', 'EXPD']
['AIG', 'EXPE']
['AIG', 'EXR']
['AIG', 'F']
['AIG', 'FAST']
['AIG', 'FBHS']
['AIG', 'FCX']
['AIG', 'FE']
['AIG', 'FFIV']
['AIG', 'FIS']
['AIG', 'FISV']
['AIG', 'FITB']
['AIG', 'FLIR']
['AIG', 'FLR']
['AIG', 'FLS']
['AIG', 'FMC']
['AIG', 'FRT']
['AIG', 'FTI']
['AIG', 'GGP']
['AIG', 'GIS']
['AIG', 'GLW']
['AIG', 'GPC']
['AIG', 'GPN']
['AIG', 'GPS']
['AIG', 'GWW']
['AIG', 'HAS']
['AIG', 'HBAN']
['AIG', 'HBI']
['AIG', 'HCP']
['AIG', 'HOG']
['AIG', 'HRB']
['AIG', 'HRL']
['AIG', 'HST']
['AIG', 'HSY']
['AIG', 'HUM']
['AIG', 'IFF']
['AIG', 'ILMN']
['AIG', 'INTU']
['AIG', 'IP']
['AIG', 'IR']
['AIG', 'IRM']
['AIG', 'IT']
['AIG', 'ITW']
['AIG', 'IVZ']
['AIG', 'JBHT']
['

...

['NWL', 'XYL']
['NWL', 'YUM']
['NWL', 'ZION']
['NWL', 'ZTS']
['OKE', 'RRC']
['OKE', 'WDC']
['OMC', 'PAYX']
['OMC', 'PBCT']
['OMC', 'PEG']
['OMC', 'PGR']
['OMC', 'PPL']
['OMC', 'PVH']
['OMC', 'PWR']
['OMC', 'PX']
['OMC', 'RCL']
['OMC', 'RE']
['OMC', 'RHT']
['OMC', 'RJF']
['OMC', 'RRC']
['OMC', 'RSG']
['OMC', 'SCG']
['OMC', 'SJM']
['OMC', 'SO']
['OMC', 'SPG']
['OMC', 'SRCL']
['OMC', 'STI']
['OMC', 'STZ']
['OMC', 'SYMC']
['OMC', 'SYY']
['OMC', 'TAP']
['OMC', 'TEL']
['OMC', 'TGT']
['OMC', 'TPR']
['OMC', 'TROW']
['OMC', 'TRV']
['OMC', 'TXT']
['OMC', 'UAL']
['OMC', 'ULTA']
['OMC', 'V']
['OMC', 'VLO']
['OMC', 'VNO']
['OMC', 'VTR']
['OMC', 'WEC']
['OMC', 'WY']
['OMC', 'XL']
['OMC', 'XYL']
['OMC', 'YUM']
['OMC', 'ZION']
['OMC', 'ZTS']
['ORLY', 'PAYX']
['ORLY', 'PBCT']
['ORLY', 'PCAR']
['ORLY', 'PCG']
['ORLY', 'PEG']
['ORLY', 'PFG']
['ORLY', 'PGR']
['ORLY', 'PH']
['ORLY', 'PHM']
['ORLY', 'PKG']
['ORLY', 'PLD']
['ORLY', 'PNR']
['ORLY', 'PNW']
['ORLY', 'PPG']
['ORLY', 'PPL']
['ORLY', 'PRGO']
['ORL

['PKG', 'VIAB']
['PKG', 'WDC']
['PKG', 'WEC']
['PKG', 'WHR']
['PKG', 'XRX']
['PLD', 'PNR']
['PLD', 'PNW']
['PLD', 'PPL']
['PLD', 'PRGO']
['PLD', 'PRU']
['PLD', 'PSA']
['PLD', 'PVH']
['PLD', 'PWR']
['PLD', 'PXD']
['PLD', 'RCL']
['PLD', 'REG']
['PLD', 'REGN']
['PLD', 'RF']
['PLD', 'RHT']
['PLD', 'RL']
['PLD', 'ROK']
['PLD', 'RRC']
['PLD', 'RSG']
['PLD', 'SCG']
['PLD', 'SEE']
['PLD', 'SHW']
['PLD', 'SJM']
['PLD', 'SNPS']
['PLD', 'SO']
['PLD', 'SPG']
['PLD', 'SRCL']
['PLD', 'STI']
['PLD', 'STT']
['PLD', 'STX']
['PLD', 'STZ']
['PLD', 'SWK']
['PLD', 'SYMC']
['PLD', 'SYY']
['PLD', 'TAP']
['PLD', 'TDG']
['PLD', 'TIF']
['PLD', 'TMK']
['PLD', 'TPR']
['PLD', 'TRIP']
['PLD', 'TRV']
['PLD', 'TSN']
['PLD', 'TSS']
['PLD', 'UAA']
['PLD', 'UAL']
['PLD', 'UDR']
['PLD', 'UHS']
['PLD', 'ULTA']
['PLD', 'UNM']
['PLD', 'V']
['PLD', 'VIAB']
['PLD', 'VNO']
['PLD', 'VRSK']
['PLD', 'VRSN']
['PLD', 'VRTX']
['PLD', 'WEC']
['PLD', 'WM']
['PLD', 'WU']
['PLD', 'XEL']
['PLD', 'XLNX']
['PLD', 'XRAY']
['PLD', 'YUM']
['P

['REG', 'RF']
['REG', 'ROK']
['REG', 'SCG']
['REG', 'SHW']
['REG', 'SO']
['REG', 'TAP']
['REG', 'TIF']
['REG', 'TRIP']
['REG', 'TRV']
['REG', 'UAL']
['REG', 'UDR']
['REG', 'UHS']
['REG', 'UNM']
['REG', 'VRTX']
['REG', 'VTR']
['REG', 'WMB']
['REG', 'XEC']
['REG', 'XEL']
['REG', 'YUM']
['REG', 'ZION']
['REGN', 'RF']
['REGN', 'RHI']
['REGN', 'RHT']
['REGN', 'RJF']
['REGN', 'RL']
['REGN', 'ROK']
['REGN', 'ROP']
['REGN', 'ROST']
['REGN', 'RRC']
['REGN', 'RSG']
['REGN', 'SBAC']
['REGN', 'SCG']
['REGN', 'SEE']
['REGN', 'SHW']
['REGN', 'SJM']
['REGN', 'SLG']
['REGN', 'SNA']
['REGN', 'SNPS']
['REGN', 'SO']
['REGN', 'SPG']
['REGN', 'SRCL']
['REGN', 'SRE']
['REGN', 'STI']
['REGN', 'STT']
['REGN', 'STX']
['REGN', 'STZ']
['REGN', 'SWK']
['REGN', 'SYMC']
['REGN', 'SYY']
['REGN', 'TAP']
['REGN', 'TDG']
['REGN', 'TEL']
['REGN', 'TGT']
['REGN', 'TIF']
['REGN', 'TMK']
['REGN', 'TPR']
['REGN', 'TRIP']
['REGN', 'TROW']
['REGN', 'TRV']
['REGN', 'TSCO']
['REGN', 'TSN']
['REGN', 'TSS']
['REGN', 'TXT']
['REGN

['SPG', 'UNM']
['SPG', 'URI']
['SPG', 'VLO']
['SPG', 'VMC']
['SPG', 'VRSK']
['SPG', 'WEC']
['SPG', 'WU']
['SPG', 'WY']
['SPG', 'WYNN']
['SPG', 'XEC']
['SPG', 'XEL']
['SPG', 'XLNX']
['SPG', 'XYL']
['SPG', 'YUM']
['SPG', 'ZION']
['SPG', 'ZTS']
['SRCL', 'STT']
['SRCL', 'STX']
['SRCL', 'TAP']
['SRCL', 'TGT']
['SRCL', 'TRIP']
['SRCL', 'TSCO']
['SRCL', 'TSN']
['SRCL', 'UAA']
['SRCL', 'UAL']
['SRCL', 'URI']
['SRCL', 'VFC']
['SRCL', 'VLO']
['SRCL', 'VMC']
['SRCL', 'VRTX']
['SRCL', 'WHR']
['SRCL', 'WU']
['SRCL', 'WYNN']
['SRCL', 'XLNX']
['SRCL', 'XYL']
['SRE', 'STI']
['SRE', 'STT']
['SRE', 'STX']
['SRE', 'SYMC']
['SRE', 'SYY']
['SRE', 'TDG']
['SRE', 'TEL']
['SRE', 'TGT']
['SRE', 'TIF']
['SRE', 'TMK']
['SRE', 'TPR']
['SRE', 'TRIP']
['SRE', 'TROW']
['SRE', 'TRV']
['SRE', 'TXT']
['SRE', 'UAL']
['SRE', 'UNM']
['SRE', 'VFC']
['SRE', 'VIAB']
['SRE', 'VNO']
['SRE', 'VRSK']
['SRE', 'VRSN']
['SRE', 'VRTX']
['SRE', 'WM']
['SRE', 'WU']
['SRE', 'WY']
['SRE', 'XL']
['SRE', 'XRX']
['SRE', 'YUM']
['SRE', 'ZIO

['UDR', 'VMC']
['UDR', 'VRSK']
['UDR', 'VRSN']
['UDR', 'VRTX']
['UDR', 'VTR']
['UDR', 'WAT']
['UDR', 'WDC']
['UDR', 'WEC']
['UDR', 'WM']
['UDR', 'WMB']
['UDR', 'WU']
['UDR', 'WY']
['UDR', 'XEC']
['UDR', 'XEL']
['UDR', 'XL']
['UDR', 'XLNX']
['UDR', 'XRAY']
['UDR', 'XRX']
['UDR', 'XYL']
['UDR', 'YUM']
['UDR', 'ZION']
['UDR', 'ZTS']
['UHS', 'XEC']
['UHS', 'XLNX']
['ULTA', 'URI']
['ULTA', 'WYNN']
['UNM', 'V']
['UNM', 'VFC']
['UNM', 'VLO']
['UNM', 'VMC']
['UNM', 'VNO']
['UNM', 'VRSK']
['UNM', 'VRSN']
['UNM', 'VRTX']
['UNM', 'VTR']
['UNM', 'WAT']
['UNM', 'WEC']
['UNM', 'WHR']
['UNM', 'WM']
['UNM', 'WU']
['UNM', 'WY']
['UNM', 'XEL']
['UNM', 'XL']
['UNM', 'XLNX']
['UNM', 'XRAY']
['UNM', 'XRX']
['UNM', 'XYL']
['UNM', 'YUM']
['UNM', 'ZION']
['UNM', 'ZTS']
['URI', 'WYNN']
['V', 'VIAB']
['V', 'VLO']
['V', 'VMC']
['V', 'VNO']
['V', 'VRSK']
['V', 'VRTX']
['V', 'VTR']
['V', 'WAT']
['V', 'WEC']
['V', 'WM']
['V', 'WU']
['V', 'WY']
['V', 'WYNN']
['V', 'XEC']
['V', 'XEL']
['V', 'XL']
['V', 'XLNX']
['V', 

In [9]:

listOfPosResult = pos_result.split('\n\n')
list11 =[]
list22 = []
listRet = []
for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    
    list11.append(list1[0])
    list22.append(list1[1])
    listRet.append(float(cumR))
    
maxVal = max(listRet)

for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    cumR = float(cumR)
    
    if(cumR==maxVal):
        print(list1[0],list1[1],round(maxVal,2))
print(len(listOfPosResult))

listOfResult = result.split('\n\n')
print(len(listOfResult))

success = (len(listOfPosResult)/len(listOfResult))*100
print('Percentage of profiting pairs in Cluster: '+ str(success)+'%')

NWL SCG 2.51
10503
23558
Percentage of profiting pairs in Cluster: 44.58358094914679%


In [13]:
listRet.sort()
print(listRet)


[6.119105640181033e-06, 3.0082713401924365e-05, 6.858945717226561e-05, 7.395548562655563e-05, 9.674215763340044e-05, 0.00010600849845204152, 0.00012722841369281924, 0.00016150536561994498, 0.00016712356761928505, 0.00016789036372188092, 0.00017807005721863335, 0.0001873172458864758, 0.0002154085871910638, 0.00021784543250305432, 0.00022795108535547114, 0.00024270471588280884, 0.00025005303544617163, 0.00025038365877477986, 0.00025919735854159856, 0.00026704676485607594, 0.000312331360104281, 0.0003949117727195528, 0.0004804949291195726, 0.0005399923645074977, 0.0005481030564138223, 0.0005535391741531726, 0.0005654598261314448, 0.00057852614684478, 0.0005793465044990853, 0.0005946175185371771, 0.0006516909758347822, 0.0006840637283436024, 0.0007271199747387414, 0.0007288639973961608, 0.0007475693628811086, 0.0007532625250524916, 0.0007736252024709245, 0.0007739742831428487, 0.0007910734130746278, 0.000803629573676261, 0.0008705470817096739, 0.000878147042015911, 0.0008997238172403854, 0